## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime

## Download

Retrieve the page

In [6]:
endpoint_slug = "C-19_IC_Cases_by_ZipCode_-_Dynamics_view_08302022"

In [7]:
endpoint_slug = "C-19_IC_Cases_by_ZipCode_-_Dynamics_view_11182022"

In [8]:
url = f"https://services7.arcgis.com/RomaVqqozKczDNgd/ArcGIS/rest/services/{endpoint_slug}/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [9]:
url

'https://services7.arcgis.com/RomaVqqozKczDNgd/ArcGIS/rest/services/C-19_IC_Cases_by_ZipCode_-_Dynamics_view_11182022/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token='

In [10]:
r = requests.get(url)

In [11]:
r

<Response [200]>

In [12]:
data = r.json()

## Parse

In [13]:
dict_list = []

In [14]:
for item in data["features"]:
    d = dict(
        county="Imperial",
        area_name=item["attributes"]["CITY"],
        zip_code=item["attributes"]["ZIPCODE"],
        confirmed_cases=item["attributes"]["TOTAL_CASES"],
    )
    dict_list.append(d)

KeyError: 'features'

In [26]:
df = pd.DataFrame(dict_list)

In [27]:
df

,county,area_name,zip_code,confirmed_cases
0,Imperial,UNKNOWN,UNKNOWN,507
1,Imperial,"BARD, CA",92222,13
2,Imperial,"BRAWLEY, CA",92227,10591
3,Imperial,"CALEXICO, CA",92231,15292
4,Imperial,"CALIPATRIA, CA",92233,1264
5,Imperial,"EL CENTRO, CA",92243,16547
6,Imperial,"EL CENTRO, CA",92244,269
7,Imperial,"HEBER, CA",92249,2447
8,Imperial,"HOLTVILLE, CA",92250,2422
9,Imperial,"IMPERIAL, CA",92251,7854


In [28]:
df["area_name"] = df["area_name"].str.replace(", CA", "").str.title()

Match zips to names

In [29]:
df["area_name"] = df["zip_code"] + ": " + df["area_name"]

In [30]:
df.loc[(df.zip_code == "92004"), "area_name"] = "92004: Borrego Springs"

In [31]:
df.loc[(df.zip_code == "UNKNOWN"), "area_name"] = "Unknown"

Get timestamp

In [32]:
date_url = f"https://services7.arcgis.com/RomaVqqozKczDNgd/ArcGIS/rest/services/{endpoint_slug}/FeatureServer/0/?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [33]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [34]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [35]:
latest_date = pd.to_datetime(timestamp).date()

In [36]:
df["county_date"] = latest_date

In [37]:
export_df = df[
    ["county", "area_name", "confirmed_cases", "county_date", "zip_code"]
].rename(columns={"area_name": "city", "zip_code": "zip"})

## Vet

In [41]:
len(export_df)

18

In [42]:
try:
    assert not len(export_df) < 18
except AssertionError:
    raise AssertionError("Imperial County's scraper is missing rows")

In [43]:
try:
    assert not len(export_df) > 18
except AssertionError:
    raise AssertionError("Imperial County's scraper has more rows than before")

## Export

In [44]:
tz = pytz.timezone("America/Los_Angeles")

In [45]:
today = datetime.now(tz).date()

In [46]:
slug = "imperial"

In [47]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [48]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [49]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"]).rename(
            columns={"area_name": "area", "city": "area"}
        )
        df["date"] = file_date
    df_list.append(df)

In [50]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [51]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)